In [1]:
pip install torch transformers

     |███████████▋                    | 321.5 MB 123.1 MB/s eta 0:00:05

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████        | 660.0 MB 134.1 MB/s eta 0:00:02

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 881.9 MB 5.2 kB/s s eta 0:00:01
     |████████████████████████████████| 3.4 MB 100.4 MB/s eta 0:00:01
     |████████████████████████████████| 764 kB 78.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 1.5 MB/s s eta 0:00:01
     |████████████████████████████████| 895 kB 82.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.3 MB 21.8 MB/s eta 0:00:01
You should consider upgrading via the '/home/default/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np

In [84]:
from transformers import ElectraForPreTraining, ElectraTokenizerFast
import torch

model = ElectraForPreTraining.from_pretrained("Seznam/small-e-czech", output_hidden_states = True)
tokenizer = ElectraTokenizerFast.from_pretrained("Seznam/small-e-czech")

In [270]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("ufal/robeczech-base")

model = AutoModelForMaskedLM.from_pretrained("ufal/robeczech-base", output_hidden_states = True)

In [266]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask, last_hidden_layers):
    #First element of model_output contains all token embeddings
    token_embeddings = torch.stack(model_output[1][-last_hidden_layers:]).mean(axis=0)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def sum_pooling(model_output, attention_mask, last_hidden_layers):
    #First element of model_output contains all token embeddings
    token_embeddings = torch.stack(model_output[1][-last_hidden_layers:]).sum(axis=0)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def concat_pooling(model_output, attention_mask, last_hidden_layers, inputs):
    #First element of model_output contains all token embeddings
    token_embeddings = torch.stack(model_output[1][-last_hidden_layers:]).reshape(inputs,30,-1)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [256]:
mats = pd.read_csv("in/tables/MATERIALS_FOR_CLASSIFICATION_CLEAN.csv")
cats = pd.read_csv("in/tables/MATERIAL_CATEGORY_FOR_CLASSIFICATION.csv")

df = pd.merge(mats,cats,on=["MATERIAL_ODS_ID","SHOP"])

def create_category_dict(df):
    return df.groupby("CATEGORY_ID").CATEGORY_PATH.unique().to_dict()
category_dict = create_category_dict(df)

# should be applied after category_decoder_dict
df = df[~df.MATERIAL_ID.duplicated()].reset_index(drop=True).copy()

for k,v in category_dict.items():
    df.loc[df.CATEGORY_ID == k,"CATEGORY_PATH_FIX"] = v[0]

In [195]:
# df = df[df.CATEGORY_PATH.str.contains("Sekačky")]

In [257]:
df.NAME_BRIEF = df.NAME_BRIEF.str.split(" ; ").apply(lambda x: " ".join(x))

In [273]:
for i in range(0,df.shape[0]+1000,1000,):
    
    df_subset = df.iloc[i:i+1000][["MATERIAL_ID","NAME_BRIEF"]].copy()

    # Tokenize sentences
    encoded_input = tokenizer(df_subset['NAME_BRIEF'].tolist(), max_length=10,
                              padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    mean_1 = mean_pooling(model_output, encoded_input['attention_mask'],1)
    sum_4 = sum_pooling(model_output, encoded_input['attention_mask'],4)
#     concat_4 = concat_pooling(model_output, encoded_input['attention_mask'],4,inputs=1000)

    df_subset["MEAN_1"] = [list(np.round(i,4)) for i in mean_1.tolist()]
    df_subset["SUM_4"] = [list(np.round(i,4)) for i in sum_4.tolist()]
#     df_subset["CONCAT_4"] = [list(np.round(i,4)) for i in concat_4.tolist()]
    
    with open("out/tables/SENT_EMBD_ROBERTA.csv", "a") as f:
                    df_subset[["MATERIAL_ID","MEAN_1","SUM_4"]].to_csv(f, index=False, header=True)
    del encoded_input, model_output, mean_1, sum_4

IndexError: list index out of range

# debug

In [250]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similiarity = cosine_similarity(embeddings)

In [251]:
df.iloc[4].NAME

'Sekačka motorová WORLD W46P'

In [252]:
df.iloc[np.argsort(cosine_similiarity[4])[::-1][:30]].NAME.values

array(['Sekačka motorová WORLD W46P', 'Sekačka motorová WORLD W45P',
       'Sekačka motorová WORLD W53 AL PRS',
       'Sekačka motorová WORLD W55 4S', 'Sekačka motorová WORLD W50P RS',
       'Sekačka motorová WORLD W50P IS', 'Sekačka motorová WORLD W46P IS',
       'Ryobi RLT3525 - 350W Elektrická strunová sekačka, šířka záběru 25cm',
       'elektrická sekačka ARM 34 06008A6101',
       'elektrická sekačka ARM 37 06008A6201',
       'AL5 51 SHQ benzínová sekačka s pojezdem',
       'akumulátor High Power 36 V - 4,0 Ah',
       ' AL5 51 SAQ benzínová sekačka s pojezdem', 'GH-PM 40P',
       '752 SXH DOV 5in1',
       'Strunová sekačka 18 V LTR 18-30 Battery (1.444-310.0) - zánovní',
       'Sekačka strunová akumulátorová GT 2025 (bez aku) - zánovní',
       'Strunová sekačka 18 V LTR 18-30 Battery (1.444-310.0) - rozbaleno',
       'Ryobi RLT5127 - 500W Elektrická strunová sekačka, šířka záběru 27cm',
       'Kartáče na kola SILENO City (4030-20)', 'BC430PRO', 'OPTIMA 38 E',
       